# Algoritmo Naive Bayes

In [20]:
import os, sys
import pandas as pd

from spacy.lang.pt import Portuguese

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#### Importing helper

In [21]:
# Importing custom functions (temp)
current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)

from helpers.string_cleaner import StringCleaner as sc

### 1. Processamento Dataset

#### 1.1 Extrair dataset

In [33]:
true_ds = pd.read_csv('datasets/base/true.csv', sep=';')
fake_ds = pd.read_csv('datasets/base/fake.csv', sep=';')

In [34]:
true_ds['target'] = 0
fake_ds['target'] = 1

In [35]:
fake_df = pd.DataFrame(fake_ds)
true_df = pd.DataFrame(true_ds)

df = pd.concat([fake_df, true_ds])
df = df.drop('idx', axis=1)

# pd.set_option('display.max_colwidth', None)

#### 1.2 Limpar o dataframe

**Observações**:
* Verificar a necessidade de adicionar as siglas dos estados brasileiros como stop words.
* Verificar a necessidade de remover o conteúdo entre colchetes e parênteses.
* Verificar a necessidade de remover formatação de datas e horários
* Verificar a necessidade de remover caracteres especiais, como: cedilha, acentos, etc.

In [36]:
# stop words using spaCy
nlp_pt = Portuguese()
stopwords = nlp_pt.Defaults.stop_words
# nlp_pt.Defaults.stop_words.remove('sem')
# nlp_pt.Defaults.stop_words.add('to')

def remove_stopwords(text):
    final_text = []
    for token in text.split():
      if token.lower() not in stopwords:
          final_text.append(token)
    return " ".join(final_text)

def denoise_text(text):
    text = sc.remove_url(text)
    text = sc.remove_special_characters(text)
    text = remove_stopwords(text)

    return text.lower()

df['full_text'] = df['full_text'].apply(denoise_text)

In [37]:
df.head(20)

,full_text,target
0,katia abreu colocar expulsao moldura nao recla...,1
1,dr ray peita bolsonaro chama conservador fake ...,1
2,reinaldo azevedo desmascarado policia federal ...,1
3,relatorio assustador bndes mostra dinheiro pub...,1
4,radialista americano fala pt vendem ilusao bra...,1
5,willian waack continuara internado sirio liban...,1
6,problema cardiaco tira william waack jornal gl...,1
7,edir macedo podera disputar eleicoes presidenc...,1
8,boechat lula alma honesta governo diabo lula d...,1
9,pt lula nao dinheiro sobreviver chama moro men...,1


#### 1.3 Aplicar tokenização

**Observações**:
* Testar tokenização posteriormente com NLP.

In [9]:
def tokenize_dataframe(dataframe):
    result = []

    for item in dataframe:
        tokenized_item = word_tokenize(item, language="portuguese")

        for token in tokenized_item:
            result.append(token)

    return result

### 2. Aplicar Naive Bayes

#### 2.1 Dividindo o dataset e transformando em um array vetorizado

In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(
    df['full_text'],
    df['target'],
    random_state=0,
    test_size=0.4
)

In [74]:
# ngram_range (1,1) -> unigram
# ngram_range (2,2) -> bigram
# ngram_range (1,2) -> unigram and bigram

vectorizer = CountVectorizer(ngram_range=(1,2)).fit(X_train)

In [77]:
X_train_vectorized = vectorizer.transform(X_train)

In [13]:
# X_train_vectorized.toarray().shape

#### 2.2 Criando um modelo de Naive Bayes e o treinando com o dataset

In [78]:
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, Y_train)

MultinomialNB(alpha=0.1)

#### 2.3 Realizando uma predição

In [79]:
Y_predict = model.predict(vectorizer.transform(X_test))

#### 2.5 Observando a precisão

In [80]:
accuracy = accuracy_score(Y_test, Y_predict)*100
print(str(accuracy) + " %")

79.55246913580247 %


#### 2.6 Métricas mais detalhadas

In [81]:
report = classification_report(Y_test, Y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.98      0.83      1294
           1       0.97      0.61      0.75      1298

    accuracy                           0.80      2592
   macro avg       0.84      0.80      0.79      2592
weighted avg       0.84      0.80      0.79      2592



#### 2.4 Executando uma matrix de confusão

In [82]:
cm = confusion_matrix(Y_test, Y_predict)
print(cm)

[[1270   24]
 [ 506  792]]


#### 2.4 Testando o modelo

In [83]:
model.predict(vectorizer.transform(
    [
        "O Podemos decidiu  expulsar o deputado federal Carlos Gaguim do partido após a Polícia Federal fazer buscas a apreensões no gabinete dele na Câmara. Com isso, a legenda abre espaço para receber a senadora expulsa pelo PMDB, Katia Abreu. Por meio de nota, a legenda informou que o afastamento do parlamentar já era algo acordado entre os filiados da sigla.  “Ainda que o parlamentar tenha comunicado a conclusão de sua desfiliação para esta semana, diante dos fatos noticiados hoje, a Executiva Nacional do Podemos solicita o imediato cancelamento de sua filiação dos quadros do partido”. O partido, que no passado chegou a cogitar lançar o parlamentar como candidato ao Senado, diz que “apoia a investigação com a ampla apuração dos eventuais crimes cometidos e a consequente responsabilização dos envolvidos, para que todos sejam punidos com o máximo rigor da Lei, independentemente de posição ou cargo ocupado",
        "Vota e manda pra todos os Bolsonaristas que vc conhece, família, etc.  Peça que façam o mesmo (votar e repassar apenas pra Bolsonaristas).  Essa é uma enquete da jovem pan e o resultado vai ser divulgado a segunda feira.  Fizeram uma primeira e deu 91% Bolsonaro e 9 Lula.  Pessoas da esquerda reclamaram e a jovem pan resolveu fazer essa segunda.  A esquerda se mobilizou, divulgaram e o Lula cresceu bastante.  Nesse momento tá 64 Bolsonaro e 36 Lula.  Vamos nos mobilizar também e golear de novo.",
        "Justiça mandou devolver mais de 600 kg de cocaína para traficantes no Rio de Janeiro",
        "Justiça obrigou a devolução de mais de 550 kg de cocaína para traficantes no estado do Rio",
        "Professor de Geografia da Escola Estadual Camilo Dias faz apologia ao crime em sala de aula. Sem notar alunos gravaram ele falando coisas absurdas, uma mãe fez o seguinte relato Tô chocada, o professor é pago para dar aula de geografia e não a desmotivar  alunos de 15 anos estudar e trabalhar. Ele tá colocando na cabeça dos adolescentes que não é errado roubar e que é uma profissão.  Que traficante, prostituta e ladrão são cidadãos de bem, pais de família e que são profissionais relatou mãe qu pede providências das autoridades. E vc pais oq acham deste profissional de educação? Escute bem o que o professor fala para os alunos."
    ]
))

array([0, 1, 0, 0, 0])